In [13]:
%autoreload

from ase.io import read

from vibes.helpers.lattice_points import get_lattice_points

In [14]:
atoms = read('geometry.in.primitive', format='aims')

primitive = read('geometry.in.primitive', format='aims')
supercell = read('geometry.in.supercell', format='aims')

In [15]:
lps, elps = get_lattice_points(primitive.cell, supercell.cell)

[vibes]        .. found 32 (31) lattice points in 0.001s


In [17]:
for lps in elps:
    print(len(lps))

8
2
2
2
4
4
4
1
1
1
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
2
2
2
1
1
1
